# Enable IMAP access in your Gmail account settings

In [1]:
# https://mail.google.com/mail/u/0/#settings/fwdandpop

# Importing Libraries

In [2]:
import imaplib
import email
from collections import defaultdict
import email.header
from email.header import decode_header

# **Gmail account credentials**

In [3]:
username = "bsaivignesh123s@gmail.com"
password = "svhq xlkg myol pvkl"

# IMAP server settings

In [4]:
imap_server = "imap.gmail.com"
imap_port = 993

# Connect to IMAP server

In [5]:
mail = imaplib.IMAP4_SSL(imap_server, imap_port)
mail.login(username, password)
mail.select("inbox")

('OK', [b'111'])

# Search for all emails

In [6]:
status, response = mail.search(None, "ALL")  # Search for all emails in the inbox

# status, response = mail.search(None, 'UNSEEN')

# status, response = mail.search(None, 'FROM', 'marketing@engage.canva.com')

# Get email IDs

In [7]:
email_ids = response[0].split()

# Create a dictionary to store emails by domain

In [8]:
emails_by_domain = defaultdict(list)

# Loop through each email

In [9]:
for email_id in email_ids:
    status, msg = mail.fetch(email_id, "(RFC822)")
    raw_email = msg[0][1]

    # Parse email using the email library
    email_message = email.message_from_bytes(raw_email)  # Use the renamed module

    # Extract domain from From address
    from_addr = email_message["From"]
    domain = from_addr.split("@")[-1]

    # Decode the subject line
    subject, encoding = decode_header(email_message["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding) if encoding else subject.decode('utf-8')

    # Extract the email body
    email_body = ""
    if email_message.is_multipart():
        for part in email_message.walk():
            if part.get_content_type() == "text/plain":
                email_body = part.get_payload(decode=True).decode(part.get_content_charset() or 'utf-8')
                break
    else:
        email_body = email_message.get_payload(decode=True).decode(email_message.get_content_charset() or 'utf-8')

    # Add email to dictionary by domain
    emails_by_domain[domain].append({
    "Subject": subject,
    "From": from_addr,
    "Body": email_body,
    "ID": email_id
})

# Printing emails subject by Domain

In [17]:
for domain, emails in emails_by_domain.items():
    print(f"Domain: {domain}")
    for email in emails:
        print(f"  Subject: {email['Subject']}, From: {email['From']}")
    print("")

Domain: email.bing.com>
  Subject: Trending news: 'Shaan Na Jaye': A Billionaire Who Was, From: Microsoft Bing <hello@email.bing.com>
  Subject: Trending news: Minhaj Zaidi Story: Hyderabad Engineer,, From: Microsoft Bing <hello@email.bing.com>
  Subject: Trending news: Venezuela model Ariana Viera dies at 26 and more, From: Microsoft Bing <hello@email.bing.com>

Domain: communication.microsoft.com>
  Subject: Updates to our terms of use, From: Microsoft <msa@communication.microsoft.com>

Domain: otter.ai>
  Subject: We'd love to hear from you Sai, From: "Otter.ai" <no-reply@otter.ai>

Domain: datacamp.com>
  Subject: Challenge: Complete 1 chapter today, From: DataCamp <team@datacamp.com>
  Subject: It’s really that easy? Yes., From: DataCamp <team@datacamp.com>
  Subject: Get certified—take your career to the next level!, From: DataCamp <team@datacamp.com>
  Subject: Sai, don't get left behind, From: DataCamp <team@datacamp.com>
  Subject: Get certified and prove you're job-ready!, Fr

# Printing email address by Domain

In [11]:
for domain, emails in emails_by_domain.items():
    print(f"Domain: {domain}")
    printed = set()
    for email in emails:
        if email['From'] not in printed:
            print(f"From: {email['From']}")
            printed.add(email['From'])
    print("")


Domain: email.bing.com>
From: Microsoft Bing <hello@email.bing.com>

Domain: communication.microsoft.com>
From: Microsoft <msa@communication.microsoft.com>

Domain: otter.ai>
From: "Otter.ai" <no-reply@otter.ai>

Domain: datacamp.com>
From: DataCamp <team@datacamp.com>
From: DataCamp Insights <team@datacamp.com>
From: Amanda at DataCamp <team@datacamp.com>

Domain: tome.app>
From: Tome Team <team@tome.app>

Domain: updates.otter.ai>
From: "The Otter.ai Team" <no-reply@updates.otter.ai>

Domain: marketing.otter.ai>
From: "The Otter.ai Team" <no-reply@marketing.otter.ai>

Domain: emails.chegg.com>
From: "Chegg" <chegg@emails.chegg.com>

Domain: mailer.resume-now.com>
From: Resume-Now <support@mailer.resume-now.com>

Domain: google.com>
From: Jupiter <mail-noreply@google.com>

Domain: waveroom.com>
From: Waveroom <noreply@waveroom.com>

Domain: discover.pinterest.com>
From: Pinterest <recommendations@discover.pinterest.com>

Domain: quora.com>
From: Quora Digest <english-personalized-dige

In [12]:
sender = input("Enter sender mail to Delete: ")


Enter sender mail to Delete: no-reply@flashgetkids.com


In [13]:
# Search for emails from sender
status, response = mail.search(None, f'(FROM "{sender}")')

# Get email IDs
email_ids = response[0].split()

# Move emails to "Bin" folder
for email_id in email_ids:
    mail.store(email_id, '+X-GM-LABELS', '\\Trash')

# Expunge deleted emails
mail.expunge()
print(f"Successfully moved {len(email_ids)} emails from {sender} to Bin!")


Successfully moved 2 emails from no-reply@flashgetkids.com to Bin!


# Calculate total emails across all domains

In [14]:

total_emails_across_domains = sum(len(emails) for emails in emails_by_domain.values())

# Print emails by domain with percentage based on the total number of domains

for domain, emails in emails_by_domain.items():
    domain_percentage = (len(emails) / total_emails_across_domains) * 100
    print(f"Domain: {domain} ({domain_percentage:.2f}% )")
    

Domain: email.bing.com> (2.70% )
Domain: communication.microsoft.com> (0.90% )
Domain: otter.ai> (0.90% )
Domain: datacamp.com> (10.81% )
Domain: tome.app> (1.80% )
Domain: updates.otter.ai> (5.41% )
Domain: marketing.otter.ai> (0.90% )
Domain: emails.chegg.com> (1.80% )
Domain: mailer.resume-now.com> (3.60% )
Domain: google.com> (0.90% )
Domain: waveroom.com> (0.90% )
Domain: discover.pinterest.com> (6.31% )
Domain: quora.com> (40.54% )
Domain: ideas.pinterest.com> (3.60% )
Domain: explore.pinterest.com> (0.90% )
Domain: mail.beehiiv.com> (1.80% )
Domain: newsletter.springernature.com> (2.70% )
Domain: inspire.pinterest.com> (0.90% )
Domain: accounts.google.com> (1.80% )
Domain: amcatmail.com> (0.90% )
Domain: amcatmail.com (0.90% )
Domain: careerflow.ai> (0.90% )
Domain: resumeworded.com> (0.90% )
Domain: mautem.resumeworded.com> (2.70% )
Domain: vbithyd.ac.in> (0.90% )
Domain: youtube.com> (0.90% )
Domain: flowcv.com> (0.90% )
Domain: flashgetkids.com> (1.80% )


# Check if the Subject Or Body contains spam keywords

In [22]:
spam_keywords = [
    "click here", "win a prize", "free gift", "claim your reward", "urgent", "act now", 
    "limited time offer", "apply now", "guaranteed", "risk-free", "no cost", "100% free", 
    "money back", "cheap", "earn money", "exclusive deal", "lower rates", "get rich quick", 
    "credit score", "pre-approved", "save big", "cash bonus", "buy now", "make money", 
    "free", "win", "winner", "limited time offer"
]

In [23]:
# Check for spam emails based on subject or body
for domain, emails in emails_by_domain.items():
    for email_info in emails:
        subject = email_info["Subject"].lower()
        body = email_info.get("Body", "").lower()

        spam_found = False

        # Check for spam keywords in the subject
        if any(keyword in subject for keyword in spam_keywords):
            print(f"Domain: {domain}, Subject: {email_info['Subject']}, From: {email_info['From']}")
            spam_found = True

        # If spam is found, ask for deletion
        if spam_found:
            delete_confirmation = input("Do you want to delete this email? (Y/N): ").strip().upper()
            if delete_confirmation == "Y":
                mail.store(email_info["ID"], '+FLAGS', '\\Deleted')
                print("Email deleted.")
            else:
                print("Email not deleted.")

# Expunge deleted emails
mail.expunge()

Domain: datacamp.com>, Subject: Challenge for Sai: Complete 1 free chapter today!, From: Amanda at DataCamp <team@datacamp.com>
Do you want to delete this email? (Y/N): y
Email deleted.
Domain: waveroom.com>, Subject: Introducing Noise Removal & Making Waveroom Free for Everyone, From: Waveroom <noreply@waveroom.com>
Do you want to delete this email? (Y/N): y
Email deleted.
Domain: quora.com>, Subject: What are the most mind blowing facts about Indian politicians?, From: Quora Suggested Spaces <knowledgenwisdom-space@quora.com>
Do you want to delete this email? (Y/N): y
Email deleted.


('OK', [b'15', b'33', b'90'])

In [24]:
# Close IMAP connection
mail.close()
mail.logout()

('BYE', [b'LOGOUT Requested'])